In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from importlib import reload

import models
import plotting
import utils.dataloaders as dl
import traintest as tt
import adversarial as adv
import utils.preproc as pre

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [3]:
base_model = torch.load('SavedModels/base_model.pth').to(device)
gmm_model = torch.load('SavedModels/gmm_model.pth').to(device)

In [14]:
def test_metrics(model, device, in_loader, out_loader, thresholds=torch.linspace(.1, 1., 1000).to(device)):
    with torch.no_grad():
        model.eval()
        conf_in = []
        conf_out = []
        count = 0.
        for ((batch_idx, (data_in, _)), (_, (data_out, _))) in zip(enumerate(in_loader),enumerate(out_loader)):
            count += 1
            data_in = data_in.to(device)
            data_out = data_out.to(device)

            output_in = model(data_in).max(1)[0].exp()
            output_out = model(data_out).max(1)[0].exp()

            conf_in.append(output_in)
            conf_out.append(output_out)
        conf_in = torch.cat(conf_in)
        conf_out = torch.cat(conf_out)
        
        tp = (conf_in[:,None] > thresholds[None,:]).sum(0).float()/(count*in_loader.batch_size)
        fp = (conf_out[:,None] > thresholds[None,:]).sum(0).float()/(count*out_loader.batch_size)
        
        mmc = conf_out.mean().item()
        auroc = -np.trapz(tp.cpu().numpy(), x=fp.cpu().numpy())
        fp95 = ((conf_out > 0.95).sum().float()/(count*out_loader.batch_size)).item()
        return mmc, auroc, fp95

In [15]:
def evaluate_model(model, device, base_loader, loaders):
    metrics = []
    mmc, _, _ = test_metrics(model, device, base_loader, base_loader)
    metrics.append(['MNIST', mmc, '-', '-'])
    for (name, data_loader) in loaders:
        mmc, auroc, fp95 = test_metrics(model, device, base_loader, data_loader)
        metrics.append([name, mmc, auroc, fp95])
    df = pd.DataFrame(metrics, columns = ['DataSet', 'MMC', 'AUROC', 'FPR@95'])
    return df.set_index('DataSet')

In [26]:
AdversarialNoiseLoader = adv.create_adv_noise_loader(base_model, dl.Noise_loader, device)

In [89]:
AdversarialSampleLoader = adv.create_adv_noise_loader(base_model, dl.MNIST_test_loader, device)

In [90]:
loaders = (
[('FMNIST', dl.FMNIST_test_loader), 
 ('EMNIST', dl.EMNIST_test_loader),
 ('GrayCIFAR10', dl.GrayCIFAR10_test_loader),
 ('Noise', dl.Noise_loader),
 ('Adv. Noise', AdversarialNoiseLoader ),
 ('Adv. Sample', AdversarialSampleLoader)]
)

In [91]:
df = evaluate_model(base_model, device, dl.MNIST_test_loader, loaders)

In [92]:
df

,MMC,AUROC,FPR@95
DataSet,,,
MNIST,0.992019,-,-
FMNIST,0.692214,0.983835,0.111
EMNIST,0.853812,0.868063,0.4885
GrayCIFAR10,0.632773,0.989706,0.0572
Noise,0.385521,0.999365,0
Adv. Noise,0.996400,0.47135,0.9884
Adv. Sample,0.999999,0.4203,1


In [93]:
GMM_AdversarialNoiseLoader = adv.create_adv_noise_loader(gmm_model, dl.Noise_loader, device)
GMM_AdversarialSampleLoader = adv.create_adv_noise_loader(gmm_model, dl.MNIST_test_loader, device)

In [94]:
GMM_loaders = (
[('FMNIST', dl.FMNIST_test_loader), 
 ('EMNIST', dl.EMNIST_test_loader),
 ('GrayCIFAR10', dl.GrayCIFAR10_test_loader),
 ('Noise', dl.Noise_loader),
 ('Adv. Noise', GMM_AdversarialNoiseLoader ),
 ('Adv. Sample', GMM_AdversarialSampleLoader)]
)

In [95]:
gmm_df = evaluate_model(gmm_model, device, dl.MNIST_test_loader, loaders)

In [96]:
gmm_df

,MMC,AUROC,FPR@95
DataSet,,,
MNIST,0.885046,-,-
FMNIST,0.406117,0.989873,0
EMNIST,0.629561,0.900836,0.0104
GrayCIFAR10,0.238907,0.998022,0
Noise,0.251223,0.999849,0
Adv. Noise,0.849148,0.742982,0
Adv. Sample,0.903927,0.441005,0.0623


In [102]:
df3 = df.merge(gmm_df, right_index=True, left_index=True, suffixes=("_plain", "_gmm"))

In [103]:
df3

,MMC_plain,AUROC_plain,FPR@95_plain,MMC_gmm,AUROC_gmm,FPR@95_gmm
DataSet,,,,,,
MNIST,0.992019,-,-,0.885046,-,-
FMNIST,0.692214,0.983835,0.111,0.406117,0.989873,0
EMNIST,0.853812,0.868063,0.4885,0.629561,0.900836,0.0104
GrayCIFAR10,0.632773,0.989706,0.0572,0.238907,0.998022,0
Noise,0.385521,0.999365,0,0.251223,0.999849,0
Adv. Noise,0.996400,0.47135,0.9884,0.849148,0.742982,0
Adv. Sample,0.999999,0.4203,1,0.903927,0.441005,0.0623
